disclamer

These results are may be biased from syntetic dataset and model and pretty much experimental

If your goal is to explore how a basic, yet reasonably good forecast model behaves across different metrics, then the `AutoReg` model from `statsmodels` is a suitable choice for this exploration. Its basic form with a single lag might be a bit more sophisticated than the Persistence Model, but it provides a good balance between simplicity and capability. Given that you're more focused on understanding metric behavior rather than achieving the best forecast accuracy, it's an appropriate choice.

Here's a rundown of why it works for your purpose:

1. **Simplicity with Capability**: `AutoReg`, especially with a single lag or a few lags, is not overly complicated. It captures some memory of the series but doesn't delve deep into more complex structures or seasonality without explicitly being told to do so (via lags).

2. **Diverse Behavior**: Since `AutoReg` does have parameters (like lags) that can be adjusted, it offers a range of behaviors in its predictions, which will be interesting when studying metric responses.

3. **Transparent Mechanism**: `AutoReg` is essentially a linear model that uses past values as predictors. This transparency is useful when you want to understand why certain metric values might be high or low.

## Import libraries

In [1]:
from datetime import datetime
from typing import Union

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_absolute_percentage_error as mape
from plotly import graph_objects as go

# xkcd-styled & humor sans font installed to give semi-sarcasm graph
plt.xkcd()
%matplotlib inline

### Constants

In [2]:
# constants
R2 = 'r2'
MAE = 'mae'
MSE = 'mse'
RMSE = 'rmse'
MASE = 'mase'
MAPE = 'mape'
SMAPE = 'smape'
MBD = 'mbd'
HUNDRED_PERCENTAGE = 100
PERFECT_THRESHOLD = 0
ACCEPTABLE_THRESHOLD = .05
MODERATE_THRESHOLD = .1
HIGH_THRESHOLD = .2
VERY_HIGH_THRESHOLD = .3
EXCEEDINGLY_HIGH_THRESHOLD = 1
PERFECT = "Perfect"
VERY_ACCEPTABLE = "Very Acceptable"
ACCEPTABLE = "Acceptable"
MODERATE = "Moderate"
HIGH = "High"
VERY_HIGH = "Very High"
EXCEEDINGLY_HIGH = "Exceedingly High"
R2_PERFECT_THRESHOLD = 1
R2_VERY_ACCEPTABLE_THRESHOLD = .95
R2_ACCEPTABLE_THRESHOLD = .9
R2_MODERATE_THRESHOLD = .8
R2_HIGH_THRESHOLD = .7
R2_VERY_HIGH_THRESHOLD = .5
R2_EXCEEDINGLY_HIGH_THRESHOLD = 0
NOT_EXPLAIN_VARIABILITY = "Doesn't Explain Variablity"
WORSE_THAN_MEAN_MODEL = "Worse Than Simple Mean Model"
BETTER_THAN_NAIVE_LOW = 0.1
BETTER_THAN_NAIVE_MED = 0.5
BETTER_THAN_NAIVE_HIGH = 0.9
NAIVE_THRESHOLD = 1
WORSE_THAN_NAIVE_MODEL = "Worse Than Naive Forecast Model"
EQUIVALENT_TO_NAIVE_MODEL = "Equivalent to Naive Model"
OVERESTIMATION = 'Overestimation'
UNDERESTIMATION = 'Underestimation'

### Function helpers

In [3]:
def autoreg_predict(train: pd.Series,
                    test: pd.Series
                    ) -> pd.Series:
    '''Predict using AutoReg with lags = 1 and trend = ct'''
    # Training the AutoReg model with lags set to 1
    model = AutoReg(train, lags=1, trend='ct')
    model = model.fit()
    # Making predictions
    predictions = model.predict(start=len(train), end=len(train)+len(test)-1)
    return predictions

def offsetmodel_predict(series: pd.Series,
                        train_size: int,
                        offset: float = .01) -> pd.Series:
    '''Predict using Offset Model with default 1% distance of the range'''
    dist = offset * (series.max()-series.min())
    predictions = series[train_size:] + dist
    return predictions

def contains_zero(arr: np.ndarray) -> bool:
    '''Check whether the array contains zero'''
    return np.any(arr == 0)

def rmse(y_true: pd.Series, 
         y_pred: pd.Series) -> float:
    """Compute the Root Mean Squared Error (RMSE)."""
    return mse(y_true, y_pred) ** .5

def mase(y_true: pd.Series, 
         y_pred: pd.Series,
         y_train: pd.Series) -> float:
    """Compute the Mean Absolute Scaled Error (MASE)."""
    # Compute the mean absolute error of the forecast
    og_mae = mae(y_true, y_pred)
    # Compute the mean absolute error of the naive method on the training data
    nf_mae = mae(y_train[1:], y_train[:-1])
    return og_mae / nf_mae

def smape(y_true: pd.Series, 
          y_pred: pd.Series) -> float:
     """Compute the Symmetric Mean Absolute Percentage Error (sMAPE)."""
     numerator = np.abs(y_pred - y_true)
     denominator = (np.abs(y_pred) + np.abs(y_true))
     return 100 * np.mean(2 * numerator / denominator)

def mbd(y_true: pd.Series, 
        y_pred: pd.Series,
        epsilon: float = 1e-10) -> tuple[str, float]:
     """Compute the Mean Bias Deviation (MBD) and its bias."""
     numerator = (y_pred - y_true)
     denominator = (y_true + epsilon)
     if np.sum(numerator) > 0 and np.sum(denominator) > 0:
          bias = OVERESTIMATION
     elif np.sum(numerator) < 0 and np.sum(denominator) > 0:
          bias = UNDERESTIMATION
     elif np.sum(numerator) > 0 and np.sum(denominator) < 0:
          bias = OVERESTIMATION
     elif np.sum(numerator) < 0 and np.sum(denominator) < 0:
          bias = UNDERESTIMATION
     return bias, 100 * np.mean(numerator / denominator)

def categorize_metrics(metric: float, y_min: float, y_max: float) -> str:
     '''Categorize standard error metrics (MAE, MSE, RMSE) into 7 categories'''
     normalized_error = metric / (y_max - y_min)
     if normalized_error == PERFECT_THRESHOLD:
          return PERFECT
     elif normalized_error <= ACCEPTABLE_THRESHOLD:
          return VERY_ACCEPTABLE
     elif normalized_error <= MODERATE_THRESHOLD:
          return ACCEPTABLE
     elif normalized_error <= HIGH_THRESHOLD:
          return MODERATE
     elif normalized_error <= VERY_HIGH_THRESHOLD:
          return HIGH
     elif normalized_error <= EXCEEDINGLY_HIGH_THRESHOLD:
          return VERY_HIGH
     else:
          return EXCEEDINGLY_HIGH
     
def categorize_mse(mse: float, y_min: float, y_max: float) -> str:
     '''Categorize MSE error metrics into 6 categories'''
     # Apply square root to mse before categorizing
     rmse = mse ** .5
     return categorize_metrics(rmse, y_min, y_max)

def categorize_pe(pe: float) -> str:
     '''Categorize PEs error metrics into categories based on magnitude.'''
     if pe == PERFECT_THRESHOLD:
          return PERFECT
     elif pe <= ACCEPTABLE_THRESHOLD*HUNDRED_PERCENTAGE:
          return VERY_ACCEPTABLE
     elif pe <= MODERATE_THRESHOLD*HUNDRED_PERCENTAGE:
          return ACCEPTABLE
     elif pe <= HIGH_THRESHOLD*HUNDRED_PERCENTAGE:
          return MODERATE
     elif pe <= VERY_HIGH_THRESHOLD*HUNDRED_PERCENTAGE:
          return HIGH
     elif pe <= EXCEEDINGLY_HIGH_THRESHOLD*HUNDRED_PERCENTAGE:
          return VERY_HIGH
     else:
          return EXCEEDINGLY_HIGH

def categorize_r2(r2: float) -> str:
     '''Categorize R2 score into 9 categories'''
     if r2 == R2_PERFECT_THRESHOLD:
          return PERFECT
     elif r2 >= R2_VERY_ACCEPTABLE_THRESHOLD:
          return VERY_ACCEPTABLE
     elif r2 >= R2_ACCEPTABLE_THRESHOLD:
          return ACCEPTABLE
     elif r2 >= R2_MODERATE_THRESHOLD:
          return MODERATE
     elif r2 >= R2_HIGH_THRESHOLD:
          return HIGH
     elif r2 >= R2_VERY_HIGH_THRESHOLD:
          return VERY_HIGH
     elif r2 > R2_EXCEEDINGLY_HIGH_THRESHOLD:
          return EXCEEDINGLY_HIGH
     elif r2 == R2_EXCEEDINGLY_HIGH_THRESHOLD:
          return NOT_EXPLAIN_VARIABILITY
     else:
          return WORSE_THAN_MEAN_MODEL

def categorize_mase(mase: float) -> str:
     '''Categorize MASE into 7 categories'''
     if mase == PERFECT_THRESHOLD:
          return PERFECT
     elif mase <= BETTER_THAN_NAIVE_LOW:
          return VERY_ACCEPTABLE
     elif mase <= BETTER_THAN_NAIVE_MED:
          return ACCEPTABLE
     elif mase <= BETTER_THAN_NAIVE_HIGH:
          return MODERATE
     elif mase < NAIVE_THRESHOLD:
          return HIGH
     elif mase == NAIVE_THRESHOLD:
          return EQUIVALENT_TO_NAIVE_MODEL
     else:
          return WORSE_THAN_NAIVE_MODEL

def display_metrics(test: pd.Series,
                    pred: pd.Series,
                    train: pd.Series,
                    y_max: Union[int, float],
                    y_min: Union[int, float]):
    '''Display all the metrics in dataframe'''
    R2_val = r2_score(test, pred)
    MAE_val = mae(test, pred)
    MSE_val = mse(test, pred)
    RMSE_val = rmse(test, pred)
    MASE_val = mase(test, pred, train)
    MAPE_val = mape(test, pred)
    sMAPE_val = smape(test, pred)
    MBD_category, MBD_val = mbd(test, pred)
    metric_dict = {
        'Metric': [
            R2,
            MAE,
            MSE,
            RMSE,
            MASE,
            MAPE,
            SMAPE,
            MBD
        ],
        'Value': [
            R2_val,
            MAE_val,
            MSE_val,
            RMSE_val,
            MASE_val,
            MAPE_val,
            sMAPE_val,
            MBD_val
        ],
        'Category': [
            None if np.isnan(R2_val) else categorize_r2(R2_val),
            None if np.isnan(MAE_val) else categorize_metrics(MAE_val, y_min, y_max),
            None if np.isnan(MSE_val) else categorize_metrics(MSE_val, y_min, y_max),
            None if np.isnan(RMSE_val) else categorize_metrics(RMSE_val, y_min, y_max),
            None if np.isnan(MASE_val) else categorize_mase(MASE_val),
            None if np.isnan(MAPE_val) else categorize_pe(MAPE_val),
            None if np.isnan(sMAPE_val) else categorize_pe(sMAPE_val),
            None if np.isnan(MBD_val) else f'{categorize_pe(abs(MBD_val))} {MBD_category}'
        ]
    }
    df = pd.DataFrame.from_dict(metric_dict)
    display(df)

def display_forecast_plotly(train: pd.Series,
                            test: pd.Series,
                            pred: pd.Series):
    '''Visualization using Plotly'''
    fig = go.Figure()
    # Plotting training data
    fig.add_trace(go.Scatter(x=train.index, y=train.values,
                        mode='lines',
                        name='Train'))
    # Plotting test data
    fig.add_trace(go.Scatter(x=test.index, y=test.values,
                        mode='markers',
                        name='Test',
                        marker=dict(color='lightgreen')))
    # Plotting prediction data
    fig.add_trace(go.Scatter(x=pred.index, y=pred.values,
                        mode='markers',
                        name='Prediction',
                        marker=dict(color='purple')))
    # Show figure
    fig.show()

def save_forecast_plotly(train: pd.Series,
                         test: pd.Series,
                         pred: pd.Series,
                         filename: str):
    '''Save visualization using Plotly as HTML file'''
    fig = go.Figure()
    # Plotting training data
    fig.add_trace(go.Scatter(x=train.index, y=train.values,
                             mode='lines',
                             name='Train'))
    # Plotting test data
    fig.add_trace(go.Scatter(x=test.index, y=test.values,
                             mode='markers',
                             name='Test',
                             marker=dict(color='lightgreen')))
    # Plotting prediction data
    fig.add_trace(go.Scatter(x=pred.index, y=pred.values,
                             mode='markers',
                             name='Prediction',
                             marker=dict(color='purple')))
    # Save figure as HTML
    fig.write_html(filename)

## Based on test size

### Small

Small: Few data points might cause magnified errors in some metrics. (e.g., a test size of just 1 or 2 data points)

#### Test size = 1

##### Cos Dataset

In [4]:
# Generate a synthetic dataset
days = 100_000
date_range = pd.date_range(datetime(1970, 1, 1), periods=days)
y = np.cos(np.linspace(0, 2 * np.pi, days))
cos_df = pd.DataFrame({'Value': y})
cos_df.set_index(date_range, inplace=True)
cos_series = cos_df['Value']

###### AutoReg Model

In [5]:
# Splitting the dataset into train and test sets
train_size = len(cos_series)-1
train, test = cos_series[:train_size], cos_series[train_size:]

predictions = autoreg_predict(train, test)

# Calculate metrics
y_max, y_min = train.max(), train.min()
display_metrics(test, predictions, train, y_max, y_min)

# Visualization using plotly
# display_forecast_plotly(train, test, predictions)
# or Save visualization using Plotly as HTML file
save_forecast_plotly(train, test, predictions, 'plots/testsize_s1_cos_ar.html')

/home/retiago/.pyenv/versions/3.11.1/envs/rf_metrics/lib/python3.11/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


,Metric,Value,Category
0,r2,NaN,None
1,mae,6.000085e-05,Very Acceptable
2,mse,3.600102e-09,Very Acceptable
3,rmse,6.000085e-05,Very Acceptable
4,mase,1.499991e+00,Worse Than Naive Forecast Model
5,mape,6.000085e-05,Very Acceptable
6,smape,5.999905e-03,Very Acceptable
7,mbd,6.000085e-03,Very Acceptable Overestimation


###### Offset Model

In [6]:
predictions = offsetmodel_predict(cos_series, train_size)

# Calculate metrics
display_metrics(test, predictions, train, y_max, y_min)

# Visualization using plotly
# display_forecast_plotly(train, test, predictions)
# or Save visualization using Plotly as HTML file
save_forecast_plotly(train, test, predictions, 'plots/testsize_s1_cos_om.html')

/home/retiago/.pyenv/versions/3.11.1/envs/rf_metrics/lib/python3.11/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.



,Metric,Value,Category
0,r2,NaN,None
1,mae,0.020000,Very Acceptable
2,mse,0.000400,Very Acceptable
3,rmse,0.020000,Very Acceptable
4,mase,499.990000,Worse Than Naive Forecast Model
5,mape,0.020000,Very Acceptable
6,smape,1.980198,Very Acceptable
7,mbd,2.000000,Very Acceptable Overestimation


##### Sin Dataset

In [7]:
# Generate a synthetic dataset
y = np.sin(np.linspace(0, 2 * np.pi, days))
sin_df = pd.DataFrame({'Value': y})
sin_df.set_index(date_range, inplace=True)
sin_series = sin_df['Value']

###### AutoReg Model

In [8]:
# Splitting the dataset into train and test sets
train_size = len(sin_series)-1
train, test = sin_series[:train_size], sin_series[train_size:]

predictions = autoreg_predict(train, test)

# Calculate metrics
y_max, y_min = train.max(), train.min()
display_metrics(test, predictions, train, y_max, y_min)

# Visualization using plotly
# display_forecast_plotly(train, test, predictions)
# or Save visualization using Plotly as HTML file
save_forecast_plotly(train, test, predictions, 'plots/testsize_s1_sin_ar.html')

/home/retiago/.pyenv/versions/3.11.1/envs/rf_metrics/lib/python3.11/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.



,Metric,Value,Category
0,r2,NaN,None
1,mae,6.284282e-05,Very Acceptable
2,mse,3.949220e-09,Very Acceptable
3,rmse,6.284282e-05,Very Acceptable
4,mase,1.571064e+00,Worse Than Naive Forecast Model
5,mape,2.565753e+11,Exceedingly High
6,smape,2.000000e+02,Exceedingly High
7,mbd,-6.284298e+07,Exceedingly High Underestimation


###### Offset Model

In [9]:
predictions = offsetmodel_predict(sin_series, train_size)

# Calculate metrics
display_metrics(test, predictions, train, y_max, y_min)

# Visualization using plotly
# display_forecast_plotly(train, test, predictions)
# or Save visualization using Plotly as HTML file
save_forecast_plotly(train, test, predictions, 'plots/testsize_s1_sin_om.html')

/home/retiago/.pyenv/versions/3.11.1/envs/rf_metrics/lib/python3.11/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.



,Metric,Value,Category
0,r2,NaN,None
1,mae,2.000000e-02,Very Acceptable
2,mse,4.000000e-04,Very Acceptable
3,rmse,2.000000e-02,Very Acceptable
4,mase,4.999979e+02,Worse Than Naive Forecast Model
5,mape,8.165620e+13,Exceedingly High
6,smape,2.000000e+02,Exceedingly High
7,mbd,2.000005e+10,Exceedingly High Overestimation


#### Test size = 2

##### Cos Dataset

###### AutoReg Model

In [10]:
# Splitting the dataset into train and test sets
train_size = len(cos_series)-2
train, test = cos_series[:train_size], cos_series[train_size:]

predictions = autoreg_predict(train, test)

# Calculate metrics
y_max, y_min = train.max(), train.min()
display_metrics(test, predictions, train, y_max, y_min)

# Visualization using plotly
# display_forecast_plotly(train, test, predictions)
# or Save visualization using Plotly as HTML file
save_forecast_plotly(train, test, predictions, 'plots/testsize_s2_cos_ar.html')

,Metric,Value,Category
0,r2,-9.239435e+09,Worse Than Simple Mean Model
1,mae,9.000153e-05,Very Acceptable
2,mse,9.000409e-09,Very Acceptable
3,rmse,9.487049e-05,Very Acceptable
4,mase,2.249971e+00,Worse Than Naive Forecast Model
5,mape,9.000153e-05,Very Acceptable
6,smape,8.999703e-03,Very Acceptable
7,mbd,9.000153e-03,Very Acceptable Overestimation


###### Offset Model

In [11]:
predictions = offsetmodel_predict(cos_series, train_size)

# Calculate metrics
display_metrics(test, predictions, train, y_max, y_min)

# Visualization using plotly
# display_forecast_plotly(train, test, predictions)
# or Save visualization using Plotly as HTML file
save_forecast_plotly(train, test, predictions, 'plots/testsize_s2_cos_om.html')

,Metric,Value,Category
0,r2,-4.106229e+14,Worse Than Simple Mean Model
1,mae,2.000000e-02,Very Acceptable
2,mse,4.000000e-04,Very Acceptable
3,rmse,2.000000e-02,Very Acceptable
4,mase,4.999850e+02,Worse Than Naive Forecast Model
5,mape,2.000000e-02,Very Acceptable
6,smape,1.980198e+00,Very Acceptable
7,mbd,2.000000e+00,Very Acceptable Overestimation


##### Sin Dataset

###### AutoReg Model

In [12]:
# Splitting the dataset into train and test sets
train_size = len(sin_series)-2
train, test = sin_series[:train_size], sin_series[train_size:]

predictions = autoreg_predict(train, test)

# Calculate metrics
y_max, y_min = train.max(), train.min()
display_metrics(test, predictions, train, y_max, y_min)

# Visualization using plotly
# display_forecast_plotly(train, test, predictions)
# or Save visualization using Plotly as HTML file
save_forecast_plotly(train, test, predictions, 'plots/testsize_s2_sin_ar.html')

,Metric,Value,Category
0,r2,-9.005022e+00,Worse Than Simple Mean Model
1,mae,9.427239e-05,Very Acceptable
2,mse,9.874759e-09,Very Acceptable
3,rmse,9.937182e-05,Very Acceptable
4,mase,2.356813e+00,Worse Than Naive Forecast Model
5,mape,2.565975e+11,Exceedingly High
6,smape,1.333389e+02,Exceedingly High
7,mbd,-6.284836e+07,Exceedingly High Underestimation


###### Offset Model

In [13]:
predictions = offsetmodel_predict(sin_series, train_size)

# Calculate metrics
display_metrics(test, predictions, train, y_max, y_min)

# Visualization using plotly
# display_forecast_plotly(train, test, predictions)
# or Save visualization using Plotly as HTML file
save_forecast_plotly(train, test, predictions, 'plots/testsize_s2_sin_om.html')

,Metric,Value,Category
0,r2,-4.052756e+05,Worse Than Simple Mean Model
1,mae,2.000000e-02,Very Acceptable
2,mse,4.000000e-04,Very Acceptable
3,rmse,2.000000e-02,Very Acceptable
4,mase,5.000007e+02,Worse Than Naive Forecast Model
5,mape,4.082810e+13,Exceedingly High
6,smape,2.000000e+02,Exceedingly High
7,mbd,1.000001e+10,Exceedingly High Overestimation


### Mid

Mid: A moderate amount of data points. (e.g., test size of 50 data points)

#### Cos Dataset

##### AutoReg Model

In [14]:
# Splitting the dataset into train and test sets
train_size = len(cos_series)-50
train, test = cos_series[:train_size], cos_series[train_size:]

predictions = autoreg_predict(train, test)

# Calculate metrics
y_max, y_min = train.max(), train.min()
display_metrics(test, predictions, train, y_max, y_min)

# Visualization using plotly
# display_forecast_plotly(train, test, predictions)
# or Save visualization using Plotly as HTML file
save_forecast_plotly(train, test, predictions, 'plots/testsize_mid_cos_ar.html')

,Metric,Value,Category
0,r2,-1.484492e+06,Worse Than Simple Mean Model
1,mae,1.530235e-03,Very Acceptable
2,mse,3.092665e-06,Very Acceptable
3,rmse,1.758597e-03,Very Acceptable
4,mase,3.823640e+01,Worse Than Naive Forecast Model
5,mape,1.530236e-03,Very Acceptable
6,smape,1.528691e-01,Very Acceptable
7,mbd,1.530236e-01,Very Acceptable Overestimation


##### Offset Model

In [15]:
predictions = offsetmodel_predict(cos_series, train_size)

# Calculate metrics
display_metrics(test, predictions, train, y_max, y_min)

# Visualization using plotly
# display_forecast_plotly(train, test, predictions)
# or Save visualization using Plotly as HTML file
save_forecast_plotly(train, test, predictions, 'plots/testsize_mid_cos_om.html')

,Metric,Value,Category
0,r2,-1.920017e+08,Worse Than Simple Mean Model
1,mae,2.000000e-02,Very Acceptable
2,mse,4.000000e-04,Very Acceptable
3,rmse,2.000000e-02,Very Acceptable
4,mase,4.997456e+02,Worse Than Naive Forecast Model
5,mape,2.000003e-02,Very Acceptable
6,smape,1.980201e+00,Very Acceptable
7,mbd,2.000003e+00,Very Acceptable Overestimation


#### Sin Dataset

##### AutoReg Model

In [16]:
# Splitting the dataset into train and test sets
train_size = len(sin_series)-50
train, test = sin_series[:train_size], sin_series[train_size:]

predictions = autoreg_predict(train, test)

# Calculate metrics
y_max, y_min = train.max(), train.min()
display_metrics(test, predictions, train, y_max, y_min)

# Visualization using plotly
# display_forecast_plotly(train, test, predictions)
# or Save visualization using Plotly as HTML file
save_forecast_plotly(train, test, predictions, 'plots/testsize_mid_sin_ar.html')

,Metric,Value,Category
0,r2,-3.158856e+00,Worse Than Simple Mean Model
1,mae,1.609285e-03,Very Acceptable
2,mse,3.419212e-06,Very Acceptable
3,rmse,1.849111e-03,Very Acceptable
4,mase,4.024321e+01,Worse Than Naive Forecast Model
5,mape,2.576633e+11,Exceedingly High
6,smape,7.940848e+01,Very High
7,mbd,-6.310910e+07,Exceedingly High Underestimation


##### Offset Model

In [17]:
predictions = offsetmodel_predict(sin_series, train_size)

# Calculate metrics
display_metrics(test, predictions, train, y_max, y_min)

# Visualization using plotly
# display_forecast_plotly(train, test, predictions)
# or Save visualization using Plotly as HTML file
save_forecast_plotly(train, test, predictions, 'plots/testsize_mid_sin_om.html')

,Metric,Value,Category
0,r2,-4.855280e+02,Worse Than Simple Mean Model
1,mae,2.000000e-02,Very Acceptable
2,mse,4.000000e-04,Very Acceptable
3,rmse,2.000000e-02,Very Acceptable
4,mase,5.001378e+02,Worse Than Naive Forecast Model
5,mape,1.633124e+12,Exceedingly High
6,smape,2.000000e+02,Exceedingly High
7,mbd,3.999981e+08,Exceedingly High Overestimation


### Large

Large: Many data points which can sometimes mask large individual errors. (e.g., test size of 10,000 data points)

#### Cos Dataset

##### AutoReg Model

In [18]:
# Splitting the dataset into train and test sets
train_size = len(cos_series)-10_000
train, test = cos_series[:train_size], cos_series[train_size:]

predictions = autoreg_predict(train, test)

# Calculate metrics
y_max, y_min = train.max(), train.min()
display_metrics(test, predictions, train, y_max, y_min)

# Visualization using plotly
# display_forecast_plotly(train, test, predictions)
# or Save visualization using Plotly as HTML file
save_forecast_plotly(train, test, predictions, 'plots/testsize_l_cos_ar.html')

,Metric,Value,Category
0,r2,-48.682553,Worse Than Simple Mean Model
1,mae,0.334105,Moderate
2,mse,0.162596,Acceptable
3,rmse,0.403232,High
4,mase,7894.197960,Worse Than Naive Forecast Model
5,mape,0.344310,Very Acceptable
6,smape,27.851009,High
7,mbd,34.430993,Very High Overestimation


##### Offset Model

In [19]:
predictions = offsetmodel_predict(cos_series, train_size)

# Calculate metrics
display_metrics(test, predictions, train, y_max, y_min)

# Visualization using plotly
# display_forecast_plotly(train, test, predictions)
# or Save visualization using Plotly as HTML file
save_forecast_plotly(train, test, predictions, 'plots/testsize_l_cos_om.html')

,Metric,Value,Category
0,r2,0.877777,Moderate
1,mae,0.020000,Very Acceptable
2,mse,0.000400,Very Acceptable
3,rmse,0.020000,Very Acceptable
4,mase,472.558077,Worse Than Naive Forecast Model
5,mape,0.021463,Very Acceptable
6,smape,2.123387,Very Acceptable
7,mbd,2.146265,Very Acceptable Overestimation


#### Sin Dataset

##### AutoReg Model

In [20]:
# Splitting the dataset into train and test sets
train_size = len(sin_series)-10_000
train, test = sin_series[:train_size], sin_series[train_size:]

predictions = autoreg_predict(train, test)

# Calculate metrics
y_max, y_min = train.max(), train.min()
display_metrics(test, predictions, train, y_max, y_min)

# Visualization using plotly
# display_forecast_plotly(train, test, predictions)
# or Save visualization using Plotly as HTML file
save_forecast_plotly(train, test, predictions, 'plots/testsize_l_sin_ar.html')

,Metric,Value,Category
0,r2,-1.060963e+01,Worse Than Simple Mean Model
1,mae,5.016213e-01,High
2,mse,3.389633e-01,Moderate
3,rmse,5.822055e-01,High
4,mase,1.323055e+04,Worse Than Naive Forecast Model
5,mape,4.163974e+11,Exceedingly High
6,smape,9.282871e+01,Very High
7,mbd,-1.019868e+08,Exceedingly High Underestimation


##### Offset Model

In [21]:
predictions = offsetmodel_predict(sin_series, train_size)

# Calculate metrics
display_metrics(test, predictions, train, y_max, y_min)

# Visualization using plotly
# display_forecast_plotly(train, test, predictions)
# or Save visualization using Plotly as HTML file
save_forecast_plotly(train, test, predictions, 'plots/testsize_l_sin_om.html')

,Metric,Value,Category
0,r2,9.862998e-01,Very Acceptable
1,mae,2.000000e-02,Very Acceptable
2,mse,4.000000e-04,Very Acceptable
3,rmse,2.000000e-02,Very Acceptable
4,mase,5.275115e+02,Worse Than Naive Forecast Model
5,mape,8.165620e+09,Exceedingly High
6,smape,1.961874e+01,Moderate
7,mbd,1.999974e+06,Exceedingly High Overestimation


## Based on number nature

### Non-zero real numbers

Non-zero real numbers: Standard scenario. (e.g., -5 and 7)

In [22]:
# Generate a synthetic dataset
days = 100
scale_factor = 10
const = 1
date_range = pd.date_range(datetime(1970, 1, 1), periods=days)
y_cos = np.cos(np.linspace(0, 2 * np.pi, days)) * scale_factor + const
y_sin = np.sin(np.linspace(0, 2 * np.pi, days)) * scale_factor + const
cos_df = pd.DataFrame({'Value': y_cos})
cos_df.set_index(date_range, inplace=True)
cos_series = cos_df['Value']
sin_df = pd.DataFrame({'Value': y_sin})
sin_df.set_index(date_range, inplace=True)
sin_series = sin_df['Value']

In [23]:
print(contains_zero(cos_series))
print(contains_zero(sin_series))

False
False


#### Cos Dataset

##### AutoReg Model

In [24]:
# Splitting the dataset into train and test sets
train_size = len(cos_series)-30
train, test = cos_series[:train_size], cos_series[train_size:]

predictions = autoreg_predict(train, test)

# Calculate metrics
y_max, y_min = train.max(), train.min()
display_metrics(test, predictions, train, y_max, y_min)

# Visualization using plotly
# display_forecast_plotly(train, test, predictions)
# or Save visualization using Plotly as HTML file
save_forecast_plotly(train, test, predictions, 'plots/num_non-zero_cos_ar.html')

,Metric,Value,Category
0,r2,-62.315761,Worse Than Simple Mean Model
1,mae,23.452731,Exceedingly High
2,mse,1046.080392,Exceedingly High
3,rmse,32.343166,Exceedingly High
4,mase,60.564513,Worse Than Naive Forecast Model
5,mape,3.057581,Very Acceptable
6,smape,115.358564,Exceedingly High
7,mbd,286.178225,Exceedingly High Overestimation


##### Offset Model

In [25]:
predictions = offsetmodel_predict(cos_series, train_size)

# Calculate metrics
display_metrics(test, predictions, train, y_max, y_min)

# Visualization using plotly
# display_forecast_plotly(train, test, predictions)
# or Save visualization using Plotly as HTML file
save_forecast_plotly(train, test, predictions, 'plots/num_non-zero_cos_om.html')

,Metric,Value,Category
0,r2,0.997580,Very Acceptable
1,mae,0.199950,Very Acceptable
2,mse,0.039980,Very Acceptable
3,rmse,0.199950,Very Acceptable
4,mase,0.516352,Moderate
5,mape,0.096379,Very Acceptable
6,smape,9.006135,Acceptable
7,mbd,4.415143,Very Acceptable Overestimation


#### Sin Dataset

##### AutoReg Model

In [26]:
# Splitting the dataset into train and test sets
train_size = len(sin_series)-30
train, test = sin_series[:train_size], sin_series[train_size:]

predictions = autoreg_predict(train, test)

# Calculate metrics
y_max, y_min = train.max(), train.min()
display_metrics(test, predictions, train, y_max, y_min)

# Visualization using plotly
# display_forecast_plotly(train, test, predictions)
# or Save visualization using Plotly as HTML file
save_forecast_plotly(train, test, predictions, 'plots/num_non-zero_sin_ar.html')

,Metric,Value,Category
0,r2,-23.700539,Worse Than Simple Mean Model
1,mae,13.065877,Very High
2,mse,252.939798,Exceedingly High
3,rmse,15.904081,Very High
4,mase,30.615358,Worse Than Naive Forecast Model
5,mape,10.455252,Moderate
6,smape,97.988246,Very High
7,mbd,322.507835,Exceedingly High Underestimation


##### Offset Model

In [27]:
predictions = offsetmodel_predict(sin_series, train_size)

# Calculate metrics
display_metrics(test, predictions, train, y_max, y_min)

# Visualization using plotly
# display_forecast_plotly(train, test, predictions)
# or Save visualization using Plotly as HTML file
save_forecast_plotly(train, test, predictions, 'plots/num_non-zero_sin_om.html')

,Metric,Value,Category
0,r2,0.996095,Very Acceptable
1,mae,0.199975,Very Acceptable
2,mse,0.039990,Very Acceptable
3,rmse,0.199975,Very Acceptable
4,mase,0.468572,Acceptable
5,mape,0.090285,Very Acceptable
6,smape,10.276412,Moderate
7,mbd,-4.050444,Very Acceptable Overestimation


### Real numbers with zeros
    
Real numbers with zeros: Important for some metrics which cannot handle zeros. (e.g., 0 and 7)

In [28]:
# Generate a synthetic dataset
days = 100
scale_factor = 10
const = 0
date_range = pd.date_range(datetime(1970, 1, 1), periods=days)
y_cos = (np.cos(np.linspace(0, 2 * np.pi, days)) * scale_factor + const).astype(int)
y_sin = (np.sin(np.linspace(0, 2 * np.pi, days)) * scale_factor + const).astype(int)
cos_df = pd.DataFrame({'Value': y_cos})
cos_df.set_index(date_range, inplace=True)
cos_series = cos_df['Value']
sin_df = pd.DataFrame({'Value': y_sin})
sin_df.set_index(date_range, inplace=True)
sin_series = sin_df['Value']

In [29]:
print(contains_zero(cos_series))
print(contains_zero(sin_series))

True
True


#### Cos Dataset

##### AutoReg Model

In [30]:
# Splitting the dataset into train and test sets
train_size = len(cos_series)-30
train, test = cos_series[:train_size], cos_series[train_size:]

predictions = autoreg_predict(train, test)

# Calculate metrics
y_max, y_min = train.max(), train.min()
display_metrics(test, predictions, train, y_max, y_min)

# Visualization using plotly
# display_forecast_plotly(train, test, predictions)
# or Save visualization using Plotly as HTML file
save_forecast_plotly(train, test, predictions, 'plots/num_with-zero_cos_ar.html')

,Metric,Value,Category
0,r2,-2.604595e+01,Worse Than Simple Mean Model
1,mae,1.447703e+01,Very High
2,mse,3.950212e+02,Exceedingly High
3,rmse,1.987514e+01,Exceedingly High
4,mase,3.995661e+01,Worse Than Naive Forecast Model
5,mape,4.722336e+14,Exceedingly High
6,smape,1.098895e+02,Exceedingly High
7,mbd,1.048569e+11,Exceedingly High Overestimation


##### Offset Model

In [31]:
predictions = offsetmodel_predict(cos_series, train_size)

# Calculate metrics
display_metrics(test, predictions, train, y_max, y_min)

# Visualization using plotly
# display_forecast_plotly(train, test, predictions)
# or Save visualization using Plotly as HTML file
save_forecast_plotly(train, test, predictions, 'plots/num_with-zero_cos_om.html')

,Metric,Value,Category
0,r2,9.975283e-01,Very Acceptable
1,mae,1.900000e-01,Very Acceptable
2,mse,3.610000e-02,Very Acceptable
3,rmse,1.900000e-01,Very Acceptable
4,mase,5.244000e-01,Moderate
5,mape,8.556839e+13,Exceedingly High
6,smape,2.505993e+01,High
7,mbd,1.900000e+10,Exceedingly High Overestimation


#### Sin Dataset

##### AutoReg Model

In [32]:
# Splitting the dataset into train and test sets
train_size = len(sin_series)-30
train, test = sin_series[:train_size], sin_series[train_size:]

predictions = autoreg_predict(train, test)

# Calculate metrics
y_max, y_min = train.max(), train.min()
display_metrics(test, predictions, train, y_max, y_min)

# Visualization using plotly
# display_forecast_plotly(train, test, predictions)
# or Save visualization using Plotly as HTML file
save_forecast_plotly(train, test, predictions, 'plots/num_with-zero_sin_ar.html')

,Metric,Value,Category
0,r2,-2.190012e+01,Worse Than Simple Mean Model
1,mae,1.221320e+01,Very High
2,mse,2.189251e+02,Exceedingly High
3,rmse,1.479612e+01,Very High
4,mase,3.121151e+01,Worse Than Naive Forecast Model
5,mape,8.175143e+15,Exceedingly High
6,smape,9.360595e+01,Very High
7,mbd,-1.815246e+12,Exceedingly High Underestimation


##### Offset Model

In [33]:
predictions = offsetmodel_predict(sin_series, train_size)

# Calculate metrics
display_metrics(test, predictions, train, y_max, y_min)

# Visualization using plotly
# display_forecast_plotly(train, test, predictions)
# or Save visualization using Plotly as HTML file
save_forecast_plotly(train, test, predictions, 'plots/num_with-zero_sin_om.html')

,Metric,Value,Category
0,r2,9.966109e-01,Very Acceptable
1,mae,1.800000e-01,Very Acceptable
2,mse,3.240000e-02,Very Acceptable
3,rmse,1.800000e-01,Very Acceptable
4,mase,4.600000e-01,Acceptable
5,mape,5.404320e+13,Exceedingly High
6,smape,1.734217e+01,Moderate
7,mbd,1.200000e+10,Exceedingly High Overestimation


### Negative numbers only

Negative numbers only: Metrics could behave differently when dealing with solely negative numbers. (e.g., -5 and -7)

### Positive numbers only

Positive numbers only: Standard scenario. (e.g., 5 and 10)

### Very small numbers

Very small numbers: Close to zero but not zero; can test for amplification of error. (e.g., 0.000001 and 0.0001)

### Very large numbers

Very large numbers: Tests for potential overflow or underflow issues in metric calculation. (e.g., 10^10 and 10^11)

## Based on Magnitude

### Large Magnitudes vs. Small Magnitudes

Large Magnitudes vs. Small Magnitudes: Examining how metrics handle data of varying scales. (e.g., 1000 vs 0.1)

### Same Magnitudes for y and yhat

Same Magnitudes for y and yhat: Tests the performance when predictions and true values are on the same scale. (e.g., 100 and 105)

### Different Magnitudes between y and yhat

Different Magnitudes between y and yhat: Investigating how metrics react when there is a noticeable scale difference between true and predicted values. (e.g., 1 vs 1000)

## Data Distribution and Patterns

### Trend

Trend: Examining cases where there's a consistent increase or decrease in data. (e.g., a series like 1, 2, 3, 4,...)

### Seasonality

Seasonality: Occurrences of regular fluctuations in data. (e.g., sinusoidal data patterns or sales spikes during the holidays)

### Random noise

Random noise: Cases where the data has a lot of unexpected fluctuations. (e.g., a largely constant sequence like 5, 5, 5, 5, 5, random noise might make it look like 5.1, 4.9, 5.3, 4.8, 5.2) [how to do this since we need the y and yhat relation not y before and y after relation]

### Outliers

Outliers: Situations where there are extreme values that might disproportionately affect metrics. (e.g., series like 1, 2, 3, 1000, 5)

### Repeated Patterns

Repeated Patterns: Situations where certain patterns in the data repeat after regular intervals. (e.g., a daily temperature dataset that consistently peaks at midday and dips at midnight like 1, 2, 3, 1, 2, 3, 1, 2, 3....)

## Nature of errors

### Systematic overestimation

Systematic overestimation: When predictions are consistently higher than true values. (e.g., true: 1, 2, 3; predicted: 3, 4, 5)

### Systematic underestimation

Systematic underestimation: When predictions are consistently lower than true values. (e.g., true: 3, 4, 5; predicted: 1, 2, 3)

### Random errors

Random errors: Unpredictable error patterns where predictions sometimes overshoot and sometimes undershoot the true values and none of the predicted values exactly match the true values. (e.g., for a true sequence of 1, 2, 3, 4, 5, a prediction might be 2, 1.5, 4, 3.5, 5.5)